In [ ]:
import pandas as pd
import numpy as np
import json
import os


import sklearn
import sklearn.svm
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt

import datasets

import joblib
import utilsLib as utils
from dotenv import load_dotenv

In [ ]:
load_dotenv()
HF_TOKEN = os.environ['HF_TOKEN']
DATASET_LINK = os.environ['DATASET_LINK']



In [ ]:
# Load encodings
ENCODING_PATH = 'utils'
ENCODINGS_NAME = {
    "subcategory":"subcategoriesAliases.json",
    "category":"categoriesAliases.json",
    "label":"labelsAliases.json",
    "type":"typesAliases.json", 
}

# Ensemble Submodules paths
MODELS_PATH = "pretrained_models"
SVC_CATEGORIES_NAME = 'categorySVCWeights.pkl'
SVC_DESCRIPTION_NAME = 'DescriptionSVM.pkl'
SVM_PARAGRAPH_NAME = 'PenaltySVM_OOV.pkl'



In [ ]:
encodings = dict()
for key in ENCODINGS_NAME:
    with open(f'{ENCODING_PATH}/{ENCODINGS_NAME[key]}', 'r') as file:
        encodings[key] = json.load(file)

In [5]:
datasetTotal = datasets.load_dataset(DATASET_LINK, token=HF_TOKEN)
datasetRaw = datasetTotal['validation'].to_pandas()

In [6]:
def baseInfoDataset(dataset, encodings, debug=False):
    subcategoriesColumn = np.zeros((dataset.shape[0], len(encodings['subcategory'].keys())), dtype=int)
    categoriesColumn = np.zeros((dataset.shape[0], len(encodings['category'].keys())), dtype=int)
    typesColumn = np.zeros((dataset.shape[0], len(encodings['type'].keys())), dtype=int)
    for i in range(dataset.shape[0]):
        try:
            subcategoriesColumn[i][encodings['subcategory'][dataset['subcategory'].iloc[i]]] = 1
        except KeyError:
            pass
        try:
            categoriesColumn[i][encodings['category'][dataset['category'].iloc[i]]] = 1
        except KeyError:
            pass
        try:
            typesColumn[i][encodings['type'][dataset['type'].iloc[i]]] = 1
        except KeyError:
            pass
    if debug:
        print(subcategoriesColumn.shape)
        print(categoriesColumn.shape)
        print(typesColumn.shape)
    
    return np.concatenate(
        (subcategoriesColumn,categoriesColumn,typesColumn),
        axis=1
    )

In [7]:
baseData = baseInfoDataset(datasetRaw, encodings)
print(baseData.shape)

(300, 133)


In [8]:
paragraphCache = {'data': None}

In [9]:
print(paragraphCache['data'])

None


In [12]:
def paragraphDataset(dataset, trainset, cache=paragraphCache):
    UNK_TOKEN = '<UNK>'
    target = dataset.copy()
    columsToEncode = ['type', 'subcategory']
    encoder = OneHotEncoder(handle_unknown='ignore')
    processor = utils.TextProcessor('utils/mostInfluentWords.json', 
                                            removed=['ˈ', 'also', 'since', 'many'],
                                            unkonwn=UNK_TOKEN)
   
    encoder.fit_transform(trainset[columsToEncode])
    encodingTS = encoder.transform(target[['type', 'subcategory']]).toarray().astype(float)
   
    if cache['data'] is None:
        paragraphs = utils.getText(target, lang='en', max_workers=10)
        cache['data'] = paragraphs
    else:
        paragraphs = cache['data']
    target['paragraph'] = paragraphs
   
    tfidfColumn = processor.process(target, column='paragraph')
    
    
    
    vocab = sorted(set(word for doc in target['paragraph'] for word in processor.tokenize(doc)))
    word_index = {word: idx for idx, word in enumerate(vocab)}
    word_index[UNK_TOKEN] = len(word_index)
    
    encodingTFIDF = np.array([processor.vectorize(doc_tfidf, word_index) for doc_tfidf in tfidfColumn])
    
    return np.hstack([encodingTFIDF, encodingTS])

In [13]:
paragraphData = paragraphDataset(datasetRaw, datasetTotal['train'].to_pandas())
print(paragraphData.shape)

100%|██████████| 300/300 [00:00<00:00, 865.77it/s] 


(300, 5310)


In [14]:
def descriptionDataset(dataset, trainset):
    UNK_TOKEN = '<UNK>'
    target = dataset.copy()
    columsToEncode = ['type', 'subcategory']
    encoder = OneHotEncoder(handle_unknown='ignore')
    processor = utils.TextProcessor(None, removed=['ˈ', 'also', 'since', 'many'],
                                    unkonwn=UNK_TOKEN)
   
    encoder.fit_transform(trainset[columsToEncode])
    encodingTS = encoder.transform(target[['type', 'subcategory']]).toarray().astype(float)
   
    tfidfColumn = processor.process(target, column='description')
    
    vocab = sorted(set(word for doc in target['description'] for word in processor.tokenize(doc)))
    word_index = {word: idx for idx, word in enumerate(vocab)}
    word_index[UNK_TOKEN] = len(word_index)
    
    encodingTFIDF = np.array([processor.vectorize(doc_tfidf, word_index) for doc_tfidf in tfidfColumn])
    
    return np.hstack([encodingTFIDF, encodingTS])
    

In [15]:
descriptionData = descriptionDataset(datasetRaw, datasetTotal['train'].to_pandas())
print(descriptionData.shape)

100%|██████████| 300/300 [00:00<00:00, 5399.28it/s]

(300, 1037)


In [19]:
labels = datasetRaw['label'].apply(lambda x: encodings['label'][x])
print(labels)

0      2
1      0
2      2
3      0
4      2
      ..
295    1
296    2
297    0
298    2
299    0
Name: label, Length: 300, dtype: int64


In [16]:
svcBaseInfo = joblib.load(f"{MODELS_PATH}/{SVC_CATEGORIES_NAME}")
svcDescription = joblib.load(f"{MODELS_PATH}/{SVC_DESCRIPTION_NAME}")
svcParagraph = joblib.load(f"{MODELS_PATH}/{SVM_PARAGRAPH_NAME}")

In [17]:
predictionsBaseInfo = svcBaseInfo.predict_proba(baseData)
predictionsParagraph = svcParagraph.predict_proba(paragraphData)
predictionsDescription = svcDescription.predict_proba(descriptionData)

AttributeError: This 'SVC' has no attribute 'predict_proba'

In [ ]:
probabilitiesFinal = np.stack((predictionsBaseInfo, predictionsParagraph, predictionsDescription), axis=2)